In [1]:
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import sentence_bleu
import torch


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# ====== 1. Load Dataset ======
dataset = load_dataset("SEACrowd/liputan6", trust_remote_code=True)
train_data = dataset["train"].shuffle(seed=42).select(range(500))
val_data = dataset["validation"].shuffle(seed=42).select(range(100))
test_data = dataset["test"].shuffle(seed=42).select(range(100))

def custom_tokenizer(text, vocab, max_len=512):
    # Tokenize the input text
    tokens = text.split()[:max_len]  # Truncate to max_len
    # Map tokens to vocab IDs, default to <OOV> if not in vocab
    token_ids = [vocab.get(token, vocab["<OOV>"]) for token in tokens]
    # Pad to max_len
    padding = [vocab["<OOV>"]] * (max_len - len(token_ids))
    return token_ids + padding


# Create a custom vocabulary with only <OOV>
vocab = {"<OOV>": 0}

# Update vocabulary with unique words from train_data
index = 1
for data in train_data["document"]:
    for word in data.split():
        if word not in vocab:
            vocab[word] = index
            index += 1

# Print the size of the vocabulary
print(f"Vocabulary Size: {len(vocab)}")



Vocabulary Size: 15687


In [3]:
# Contoh data dari train_data
sample_data = train_data["document"][:3]  # Ambil 3 sampel pertama untuk contoh

# Tokenisasi data
for i, text in enumerate(sample_data):
    tokenized = custom_tokenizer(text, vocab, max_len=512)
    print(f"Sample {i + 1}:")
    print("Original Text:", text)
    print("Tokenized IDs:", tokenized)
    print("Sequence Length (Excluding Padding):", len([t for t in tokenized if t != vocab["<OOV>"]]))
    print("Number of Paddings:", len([t for t in tokenized if t == vocab["<OOV>"]]))
    print("-" * 80)


Sample 1:
Original Text: Liputan6 . com , Jombang : Maman Sugianto alias Sugik , yang masih berstatus terdakwa kasus pembunuhan Asrori , ditangguhkan penahanannya sejak Senin ( 24/11 ) malam . Penangguhan penahanannya ini dimanfaatkan Sugik untuk berkunjung ke rumah sanak saudaranya . Kedatangan Sugik disambut penuh haru bahkan sang bibi langsung jatuh pingsan . Sugik bebas bersyarat dari Lembaga Pemasyarakatan Jombang setelah menjadi korban salah tangkap dan peradilan sesat dalam kasus pembunuhan Asrori . Terdakwa lainnya yakni Devid dan Kemat , hingga kini masih dipenjara dan tengah mengajukan peninjauan kembali ( PK ) ke Mahkamah Agung . Sugik dan istrinya berharap majelis hakim lebih bijaksana untuk membebaskan dirinya dari semua tuduhan . Sebab , ia memang merasa tak membunuh Asrori seperti yang dituduhkan polisi dan jaksa [ baca : Saksi Ahli : Terdakwa Bisa Dibebaskan ] . ( UPI/Bambang Ronggo ) .
Tokenized IDs: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 4, 11, 12, 13, 14, 15, 16, 17, 4, 18,

In [4]:

# ====== 2. Dataset Preparation ======
class CustomTokenizerDataset(Dataset):
    def __init__(self, articles, summaries, tokenizer, vocab, max_len=512):
        self.articles = articles
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.articles)

    def __getitem__(self, idx):
        article = self.articles[idx]
        summary = self.summaries[idx]

        article_ids = self.tokenizer(article, self.vocab, max_len=self.max_len)
        summary_ids = self.tokenizer(summary, self.vocab, max_len=self.max_len)

        return {
            "input_ids": torch.tensor(article_ids, dtype=torch.long),
            "target_ids": torch.tensor(summary_ids, dtype=torch.long),
        }

train_dataset = CustomTokenizerDataset(
    train_data["document"], train_data["summary"], custom_tokenizer, vocab
)
val_dataset = CustomTokenizerDataset(
    val_data["document"], val_data["summary"], custom_tokenizer, vocab
)
test_dataset = CustomTokenizerDataset(
    test_data["document"], test_data["summary"], custom_tokenizer, vocab
)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)


In [5]:

# ====== 3. Define Transformer Model ======
class Seq2SeqTransformer(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads, ff_dim, num_layers):
        super(Seq2SeqTransformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.transformer = nn.Transformer(
            d_model=embed_size,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=ff_dim,
        )
        self.fc = nn.Linear(embed_size, vocab_size)

    def forward(self, src, tgt):
        src_embed = self.embedding(src).permute(1, 0, 2)  # (seq_len, batch_size, embed_size)
        tgt_embed = self.embedding(tgt).permute(1, 0, 2)
        transformer_out = self.transformer(src_embed, tgt_embed)
        output = self.fc(transformer_out).permute(1, 0, 2)  # (batch_size, seq_len, vocab_size)
        return output


In [6]:

# ====== 4. Training and Evaluation ======
def train_model(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        target_ids = batch["target_ids"].to(device)

        optimizer.zero_grad()
        output = model(input_ids, target_ids[:, :-1])  # Teacher forcing
        loss = criterion(output.reshape(-1, output.size(-1)), target_ids[:, 1:].reshape(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)




def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            target_ids = batch["target_ids"].to(device)

            output = model(input_ids, target_ids[:, :-1])
            loss = criterion(output.reshape(-1, output.size(-1)), target_ids[:, 1:].reshape(-1))
            total_loss += loss.item()
    return total_loss / len(dataloader)



In [7]:

# Plotting training and validation loss
def plot_loss(train_losses, val_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Training and Validation Loss")
    plt.show()



In [8]:
def decode_token_ids(token_ids, vocab):
    """Decode token IDs back into text using vocab mapping."""
    # Create reverse mapping: ID to token
    id_to_token = {v: k for k, v in vocab.items()}
    # Decode tokens, ignore <OOV>
    tokens = [id_to_token[token_id] for token_id in token_ids if token_id != vocab["<OOV>"]]
    return " ".join(tokens)


def calculate_bleu(model, dataloader, vocab, device):
    """Calculate BLEU score for model predictions."""
    model.eval()
    scores = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            target_ids = batch["labels"].to(device)

            # Generate predictions (adjust based on your model output)
            outputs = model(input_ids)
            predictions = torch.argmax(outputs.logits, dim=-1).cpu().numpy()

            # Decode predictions and targets
            for pred, target in zip(predictions, target_ids.cpu().numpy()):
                pred_text = decode_token_ids(pred, vocab)
                target_text = decode_token_ids(target, vocab)
                scores.append(sentence_bleu([target_text.split()], pred_text.split(), weights=(0.5, 0.5)))

    return sum(scores) / len(scores)



In [11]:
from tqdm import tqdm

# Parameter model
vocab_size = len(vocab)
embed_size = 1024
num_heads = 8
ff_dim = 1024
num_layers = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model, optimizer, and criterion
model = Seq2SeqTransformer(vocab_size, embed_size, num_heads, ff_dim, num_layers).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=vocab["<OOV>"])


In [12]:

# Epochs and Batch Size
num_epochs = 10
batch_size = 16  # Adjust according to GPU memory

# Training loop
train_losses, val_losses = [], []

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    model.train()
    train_loss = 0
    
    # Progress bar for batches
    with tqdm(total=len(train_loader), desc="Training Batches", leave=False) as pbar:
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            target_ids = batch["target_ids"].to(device)
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(input_ids, target_ids[:, :-1])
            
            # Calculate loss and backpropagate
            loss = criterion(outputs.reshape(-1, outputs.size(-1)), target_ids[:, 1:].reshape(-1))
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            pbar.update(1)
            pbar.set_postfix({"Batch Loss": loss.item()})
    
    train_loss /= len(train_loader)
    val_loss = evaluate_model(model, val_loader, criterion, device)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    print(f"Epoch {epoch + 1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

# Plot loss
plot_loss(train_losses, val_losses)



Epoch 1/5


KeyboardInterrupt: 

In [ ]:
bleu_score = calculate_bleu(model, test_loader, custom_tokenizer, vocab, device)
print(f"BLEU Score: {bleu_score:.4f}")


BLEU Score: 0.0002


In [ ]:
def generate_summary(model, input_text, tokenizer, vocab, device, max_len=50):
    """
    Generate a summary for a given input text using the model.
    """
    model.eval()
    with torch.no_grad():
        # Tokenize the input text
        input_ids = torch.tensor(
            [tokenizer(input_text, vocab, max_len=max_len)], dtype=torch.long
        ).to(device)

        # Start token for generation
        start_token = vocab["<s>"]
        end_token = vocab["</s>"]

        # Initialize the target sequence with the start token
        target_ids = torch.tensor([[start_token]], dtype=torch.long).to(device)

        for _ in range(max_len):
            # Generate output probabilities
            output = model(input_ids, target_ids)
            next_token = torch.argmax(output[:, -1, :], dim=-1)

            # Append the next token to the target sequence
            target_ids = torch.cat([target_ids, next_token.unsqueeze(0)], dim=1)

            # Stop if end token is generated
            if next_token.item() == end_token:
                break

        # Decode the generated token IDs into text
        generated_summary = decode_token_ids(target_ids.squeeze().tolist(), vocab)
        return generated_summary

# Generate summaries for test data
def predict_summaries_original(model, dataset, tokenizer, vocab, device, num_samples=10, max_len=50):
    """
    Generate and display predicted summaries using original text input.
    """
    model.eval()
    predictions = []
    with torch.no_grad():
        for i in range(num_samples):
            # Load original text input and target
            input_text = dataset[i]["document"]
            target_text = dataset[i]["summary"]

            # Tokenize the original input text
            input_ids = torch.tensor(
                [tokenizer(input_text, vocab, max_len=max_len)], dtype=torch.long
            ).to(device)

            # Generate summary
            predicted_summary = generate_summary(
                model, input_text, tokenizer, vocab, device, max_len=max_len
            )

            predictions.append({
                "input": input_text,
                "target": target_text,
                "predicted": predicted_summary
            })

    return predictions


In [ ]:
from datasets import load_dataset

# Load the Liputan6 dataset
dataset = load_dataset("SEACrowd/liputan6", trust_remote_code=True)

# Select the test set
test_data = dataset["test"]

# Generate predictions
predictions = predict_summaries_original(model, test_data, custom_tokenizer, vocab, device, num_samples=5)

# Display predictions
for i, pred in enumerate(predictions):
    print(f"Sample {i + 1}")
    print(f"Input: {pred['input']}")
    print(f"Target: {pred['target']}")
    print(f"Predicted: {pred['predicted']}")
    print("-" * 50)


Sample 1
Input: Liputan6 . com , Bangka : Kapal patroli Angkatan Laut Republik Indonesia , Belinyu , baru-baru ini , menangkap tiga kapal nelayan berbendera Thailand , yakni KM Binatama , KM Sumber Jaya II , dan KM Mataram di Perairan Belitung Utara . Ketiga kapal itu ditangkap karena melanggar zona ekonomi ekslusif Indonesia . Saat ini , kapal-kapal itu diamankan di Pos Lanal Pelabuhan Pangkalan Balam , Bangka-Belitung . Menurut Komandan Pangkalan TNI AL Bangka Letnan Kolonel Laut Fredy Egam , selain menangkap tiga kapal , ALRI juga memeriksa 43 anak buah kapal . Mereka disergap saat sedang mengangkat jaring pukat harimau di Perairan Belitung Utara . Dari jumlah itu , hanya enam orang yang dijadikan tersangka , yakni tiga nahkoda dan tiga kepala kamar mesin kapal . Sedangkan ABK yang lain akan dideportasi ke negara asalnya . Meski berhasil menahan enam tersangka , TNI AL gagal mengamankan ikan tangkapan nelayan Thailand tersebut . Sebab , sebelum patroli datang , mereka telah memindah

In [ ]:
def inspect_tokenization(tokenizer, vocab, text, max_len=50):
    """
    Inspect the tokenization, sequence, and padding for a given text.
    """
    print("Original Text:")
    print(text)
    print("\nTokenized IDs:")
    token_ids = tokenizer(text, vocab, max_len=max_len)
    print(token_ids)

    print("\nTokenized Tokens:")
    id_to_token = {v: k for k, v in vocab.items()}
    tokens = [id_to_token.get(token_id, "<UNK>") for token_id in token_ids]
    print(tokens)

    print("\nPadded Sequence:")
    print(token_ids)

# Example usage with one sample from the dataset
sample = test_data[0]  # Replace with your dataset object
input_text = sample["document"]

inspect_tokenization(custom_tokenizer, vocab, input_text, max_len=50)


Original Text:
Liputan6 . com , Bangka : Kapal patroli Angkatan Laut Republik Indonesia , Belinyu , baru-baru ini , menangkap tiga kapal nelayan berbendera Thailand , yakni KM Binatama , KM Sumber Jaya II , dan KM Mataram di Perairan Belitung Utara . Ketiga kapal itu ditangkap karena melanggar zona ekonomi ekslusif Indonesia . Saat ini , kapal-kapal itu diamankan di Pos Lanal Pelabuhan Pangkalan Balam , Bangka-Belitung . Menurut Komandan Pangkalan TNI AL Bangka Letnan Kolonel Laut Fredy Egam , selain menangkap tiga kapal , ALRI juga memeriksa 43 anak buah kapal . Mereka disergap saat sedang mengangkat jaring pukat harimau di Perairan Belitung Utara . Dari jumlah itu , hanya enam orang yang dijadikan tersangka , yakni tiga nahkoda dan tiga kepala kamar mesin kapal . Sedangkan ABK yang lain akan dideportasi ke negara asalnya . Meski berhasil menahan enam tersangka , TNI AL gagal mengamankan ikan tangkapan nelayan Thailand tersebut . Sebab , sebelum patroli datang , mereka telah memindahk